# Wines Points prediction 

In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [2]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import cufflinks as cf; cf.go_offline()
import numpy as np

C:\Users\mor\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning:

A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2



In [ ]:
import os
os._exit(00)

In [6]:
!pip install numpy>=1.18.5,<1.25.0 --upgrade

The system cannot find the file specified.


In [218]:
import gensim
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Flatten, TextVectorization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical, plot_model, pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import activations
from sklearn.model_selection import train_test_split

In [143]:
import re 
import nltk
from nltk.corpus import stopwords

# Download stopwords for English, French, Italian, and Spanish
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mor\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [183]:
wine_reviews = pd.read_csv("C:/Users/mor/Python_ML_2023/data/winemag-data-130k-v2.csv")
wine_reviews.shape

(129971, 14)

In [184]:

## https://www.kaggle.com/datasets/zynicide/wine-reviews

In [185]:
wine_reviews.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# Cleaning the data

In [186]:
wine_reviews = wine_reviews.drop(columns = ['Unnamed: 0','region_2','taster_twitter_handle','designation'])
wine_reviews = wine_reviews.drop_duplicates()
wine_reviews.shape

(119988, 10)

In [187]:
wine_reviews.head()

,country,description,points,price,province,region_1,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [188]:
null_counts = wine_reviews.isnull().sum()
null_counts.sort_values(ascending=False)

taster_name    24917
region_1       19560
price           8395
country           59
province          59
variety            1
description        0
points             0
title              0
winery             0
dtype: int64

In [189]:
# Replace null values in "taster_name" and "region_1" to 'Unknown'

wine_reviews['taster_name'].fillna('Unknown', inplace=True)
wine_reviews['region_1'].fillna('Unknown', inplace=True)

# Remove null values from variety and country- Only 59

wine_reviews= wine_reviews.dropna(subset=['country'])
wine_reviews= wine_reviews.dropna(subset=['variety'])

# Calculate the mean price by province and replace null in price by the mean

average_prices = wine_reviews.groupby('province')['price'].transform('mean')
wine_reviews['price'].fillna(average_prices, inplace=True)

In [190]:
null_price_records = wine_reviews[wine_reviews['price'].isnull()]

# Print the records with null price
null_price_records

,country,description,points,price,province,region_1,taster_name,title,variety,winery
16766,South Africa,Winemaker: Charles Hopkins. There are some mea...,88,NaN,Ceres Plateau,Unknown,Lauren Buzzeo,De Grendel 2015 Cape Winemakers Guild Op Die B...,Pinot Noir,De Grendel
34865,South Africa,Winemaker: Neil Ellis. A blend of 70% Temprani...,91,NaN,Piekenierskloof,Unknown,Lauren Buzzeo,Neil Ellis 2013 Cape Winemakers Guild Insigniu...,Red Blend,Neil Ellis
110246,Egypt,"Ripe pear, apple and tropical fruit in the nos...",84,NaN,Egypt,Unknown,Susan Kostrzewa,Sahara Vineyards 2008 Caspar Blanc de Noirs Gr...,Grenache,Sahara Vineyards


In [191]:
# Calculate the mean price by country and replace the remaining null in price by the mean

average_prices_c = wine_reviews.groupby('country')['price'].transform('mean')
wine_reviews['price'].fillna(average_prices_c, inplace=True)

null_price_records = wine_reviews[wine_reviews['price'].isnull()]

# Print the records with null price
null_price_records

,country,description,points,price,province,region_1,taster_name,title,variety,winery
110246,Egypt,"Ripe pear, apple and tropical fruit in the nos...",84,NaN,Egypt,Unknown,Susan Kostrzewa,Sahara Vineyards 2008 Caspar Blanc de Noirs Gr...,Grenache,Sahara Vineyards


In [192]:
# Removing the last null

wine_reviews= wine_reviews.dropna(subset=['price'])

In [193]:
# Extract year out of the title and add to the data

def extract_year(title):
    for word in title.split():
        if word.isdigit():
            if 1900 <= int(word) <= 2017:
                return word
        # elif word == 'NV':
        #     return word
        
    return None

#add Year Column
wine_reviews['year'] = wine_reviews['title'].apply(lambda i: extract_year(i))
wine_reviews.head(1)

,country,description,points,price,province,region_1,taster_name,title,variety,winery,year
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,27.76669,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013


In [194]:
null_counts = wine_reviews.year.isnull().sum()
null_counts

4283

In [195]:
# Calculate the mean year by country and replace the null in year by the mean

wine_reviews['year'] = pd.to_numeric(wine_reviews['year'], errors='coerce')
average_prices_y = wine_reviews.groupby('country')['year'].transform('mean')
wine_reviews['year'].fillna(average_prices_y, inplace=True)

wine_reviews['year'] = wine_reviews['year'].astype(int)

null_year_records = wine_reviews[wine_reviews['year'].isnull()]

# Print the records with null
null_year_records

,country,description,points,price,province,region_1,taster_name,title,variety,winery,year


In [196]:
#Removing title
wine_reviews = wine_reviews.drop(columns = ['title'])

In [197]:
wine_reviews.head(3)

,country,description,points,price,province,region_1,taster_name,variety,winery,year
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,27.76669,Sicily & Sardinia,Etna,Kerin O’Keefe,White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.00000,Douro,Unknown,Roger Voss,Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.00000,Oregon,Willamette Valley,Paul Gregutt,Pinot Gris,Rainstorm,2013


In [198]:
# Assuming the dataset includes English, French, and Spanish
nltk.download('stopwords')
stop_words = set(stopwords.words('english') + stopwords.words('french')+ stopwords.words('italian') + stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [199]:
def clean_description(description):
    # Remove numbers, special characters, and links
    description = re.sub(r'\d+', '', description)  # Remove numbers
    description = re.sub(r'[^\w\s]', '', description)  # Remove special characters
    description = re.sub(r'http\S+|www.\S+', '', description)  # Remove links

    # Tokenize the description into words
    words = nltk.word_tokenize(description)

    # Remove stop words
    words = [word for word in words if word.lower() not in stop_words]

    # Join the words back into a cleaned description
    cleaned_description = ' '.join(words)
    
    return cleaned_description

In [200]:
# Clean the 'description' column
wine_reviews['cleaned_descriptions'] = wine_reviews['description'].apply(clean_description)

In [201]:
#Removing description
wine_reviews = wine_reviews.drop(columns = ['description'])
wine_reviews['description_len'] = wine_reviews['cleaned_descriptions'].str.split().str.len()
wine_reviews.head(3)

,country,points,price,province,region_1,taster_name,variety,winery,year,cleaned_descriptions,description_len
0,Italy,87,27.76669,Sicily & Sardinia,Etna,Kerin O’Keefe,White Blend,Nicosia,2013,Aromas include tropical fruit broom brimstone ...,21
1,Portugal,87,15.00000,Douro,Unknown,Roger Voss,Portuguese Red,Quinta dos Avidagos,2011,ripe fruity wine smooth still structured Firm ...,20
2,US,87,14.00000,Oregon,Willamette Valley,Paul Gregutt,Pinot Gris,Rainstorm,2013,Tart snappy flavors lime flesh rind dominate g...,17


# Target, train and test

In [202]:
categorical_features =  ['country','province','region_1','variety','winery','year']
numerical_features = ['price']
description_features = ['cleaned_descriptions']

In [203]:
wine_reviews_dl = wine_reviews[categorical_features+numerical_features+description_features+['points']]
np.where(wine_reviews_dl=='NV')
indices = np.where(wine_reviews_dl == 'NV')
wine_reviews_dl = wine_reviews_dl.drop(indices[0])
wine_reviews_dl.shape

(119927, 9)

In [204]:
# Split the data into train and test sets
wine_reviews_X = wine_reviews_dl.drop(columns='points').head(10000)
wine_reviews_y = wine_reviews_dl['points'].head(10000)

X_train, X_test, y_train, y_test = train_test_split(wine_reviews_X, wine_reviews_y, test_size=0.2, random_state=17)

np.where(X_test=='NV')

(array([], dtype=int64), array([], dtype=int64))

In [205]:
# Data processing
for feature in categorical_features:
    wine_reviews_X[feature] = pd.Categorical(wine_reviews_X[feature]).codes.astype('float32')
    X_train[feature] = pd.Categorical(X_train[feature]).codes.astype('float32')
    X_test[feature] = pd.Categorical(X_test[feature]).codes.astype('float32')
    
wine_reviews_X.isnull().sum().sum()
wine_reviews_X.winery.isnull().sum().sum()

0

In [206]:
unique_values_winery = X_train['winery'].unique()
unique_values_variety = X_train['variety'].unique()
unique_values_country = X_train['country'].unique()
unique_values_year = X_train['year'].unique()


print(unique_values_winery.shape)
print(unique_values_variety.shape)
print(unique_values_country.shape)
print(unique_values_year.shape)

(4760,)
(314,)
(34,)
(28,)


In [207]:
wine_reviews_y

0        87
1        87
2        87
3        87
4        87
         ..
10040    89
10041    89
10042    89
10043    89
10044    89
Name: points, Length: 10000, dtype: int64

In [211]:
# MinMaxScaler
print(wine_reviews_y.shape)  # Check the shape before applying MinMaxScaler

# Reshape to a 2D array if needed
wine_reviews_y = wine_reviews_y.reshape(-1, 1)

# MinMaxScaler
minmax = MinMaxScaler()
wine_reviews_y_scaled = minmax.fit_transform(wine_reviews_y)

(1, 10000)


0

In [212]:
# Find the maximum description length
max_desc_len = wine_reviews['description_len'].max()

print("Maximum Description Length:", max_desc_len)

Maximum Description Length: 76


In [220]:
max_words = 1024  # Maximum number of words to consider

tokenizer_1024 = Tokenizer(num_words=1024)
tokenizer_1024.fit_on_texts(X_train.cleaned_descriptions)
desc_1024 = tokenizer_1024.texts_to_sequences(X_train.cleaned_descriptions)
max_seq_length = max(len(seq) for seq in desc_1024)

desc_1024_pad = pad_sequences(desc_1024, truncating='post', maxlen=max_seq_length)
text_input_1 = Input(shape=(max_seq_length,))

# # Split the data into training and testing sets
# X_train_text1, X_test_text1, y_train1, y_test1 = train_test_split(
#     df, y_train, test_size=0.2, random_state=42
# )
tokenizer_1024_test = Tokenizer(num_words=1024)
tokenizer_1024_test.fit_on_texts(X_test.cleaned_descriptions)
desc_1024_test = tokenizer_1024.texts_to_sequences(X_test.cleaned_descriptions)
max_seq_length_test = max(len(seq) for seq in desc_1024_test)
# df = pd.DataFrame({'winery': X_train['winery'], 'desc_1024': desc_1024_pad})
desc_1024_pad_test = pad_sequences(desc_1024_test, truncating='post', maxlen=max_seq_length)

cat_input_1 = Input(shape=(1,))
embedding_1 = Embedding(input_dim=4720, output_dim=1)(cat_input_1)
flat_embed_1 = Flatten()(embedding_1)

# Embedding layers for textual features
embedding_dim = 128
embedding_1 = Embedding(input_dim=max_words, output_dim=embedding_dim)(text_input_1)
flatten_1 = Flatten()(embedding_1)

# Concatenate flattened embeddings with dense layers for regression
concatenated = Flatten()(flatten_1)  # Remove extra Flatten layer
x = Dense(256, activation='relu')(concatenated)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='linear')(x)  # Use linear activation for continuous target variable

# Define the model
model = Model(inputs=[text_input_1], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=MeanSquaredError(),
              metrics=[MeanAbsoluteError()])

# Train the model
model.fit([desc_1024_pad], y_train,
          batch_size=32,
          epochs=10,
          validation_data=([desc_1024_pad_test], y_test))

# Evaluate the model
loss, mae = model.evaluate([desc_1024_pad_test], y_test)
print('Test Loss:', loss)
print('Test MAE:', mae)


Epoch 1/10

250/250 [==============================] - 9s 28ms/step - loss: 400.1316 - mean_absolute_error: 9.0369 - val_loss: 6.7610 - val_mean_absolute_error: 2.0523
Epoch 2/10
250/250 [==============================] - 7s 26ms/step - loss: 5.3225 - mean_absolute_error: 1.7889 - val_loss: 4.9583 - val_mean_absolute_error: 1.7747
Epoch 3/10
250/250 [==============================] - 7s 28ms/step - loss: 3.6906 - mean_absolute_error: 1.5124 - val_loss: 4.6144 - val_mean_absolute_error: 1.7099
Epoch 4/10
250/250 [==============================] - 7s 28ms/step - loss: 3.0287 - mean_absolute_error: 1.3766 - val_loss: 5.6029 - val_mean_absolute_error: 1.8563
Epoch 5/10
250/250 [==============================] - 7s 28ms/step - loss: 2.3855 - mean_absolute_error: 1.2233 - val_loss: 5.0826 - val_mean_absolute_error: 1.7667
Epoch 6/10
250/250 [==============================] - 7s 28ms/step - loss: 1.8786 - mean_absolute_error: 1.0757 - val_loss: 5.2918 - val_mean_absolute_error: 1.8218
Epoch 7

In [223]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [234]:
max_words = 1024  # Maximum number of words to consider

# Tokenize training set descriptions
tokenizer_1024 = Tokenizer(num_words=max_words)
tokenizer_1024.fit_on_texts(X_train.cleaned_descriptions)
desc_1024 = tokenizer_1024.texts_to_sequences(X_train.cleaned_descriptions)
max_seq_length = max(len(seq) for seq in desc_1024)
desc_1024_pad = pad_sequences(desc_1024, truncating='post', padding='post', maxlen=max_seq_length)
text_input_1 = Input(shape=(max_seq_length,))

# Tokenize test set descriptions using the same tokenizer
desc_1024_test = tokenizer_1024.texts_to_sequences(X_test.cleaned_descriptions)
desc_1024_pad_test = pad_sequences(desc_1024_test, truncating='post', padding='post', maxlen=max_seq_length)

cat_input_1 = Input(shape=(1,))
embedding_1 = Embedding(input_dim=len(tokenizer_1024.word_index) + 1, output_dim=16)(cat_input_1)
flat_embed_1 = Flatten()(embedding_1)

embedding_dim = 128
embedding_1_text = Embedding(input_dim=max_words, output_dim=embedding_dim)(text_input_1)
flatten_1 = Flatten()(embedding_1_text)

concatenated = Concatenate()([flatten_1, flat_embed_1])
x = Dense(256, activation='relu')(concatenated)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='linear')(x)

model = Model(inputs=[text_input_1, cat_input_1], outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=MeanSquaredError(),
              metrics=[MeanAbsoluteError()])

model.fit([desc_1024_pad, X_train['winery']], y_train,
          batch_size=32,
          epochs=10,
          validation_data=([desc_1024_pad_test, X_test['winery']], y_test))

loss, mae = model.evaluate([desc_1024_pad_test, X_test['winery']], y_test)
print('Test Loss:', loss)
print('Test MAE:', mae)


Epoch 1/10
250/250 [==============================] - 8s 29ms/step - loss: 492.6464 - mean_absolute_error: 10.3147 - val_loss: 7.7701 - val_mean_absolute_error: 2.1548
Epoch 2/10
250/250 [==============================] - 7s 28ms/step - loss: 5.3521 - mean_absolute_error: 1.7786 - val_loss: 5.2462 - val_mean_absolute_error: 1.8074
Epoch 3/10
250/250 [==============================] - 7s 29ms/step - loss: 2.9092 - mean_absolute_error: 1.3094 - val_loss: 5.9066 - val_mean_absolute_error: 1.9266
Epoch 4/10
250/250 [==============================] - 7s 29ms/step - loss: 1.7252 - mean_absolute_error: 0.9953 - val_loss: 6.5521 - val_mean_absolute_error: 2.0293
Epoch 5/10
250/250 [==============================] - 7s 28ms/step - loss: 1.1976 - mean_absolute_error: 0.8180 - val_loss: 7.1020 - val_mean_absolute_error: 2.1184
Epoch 6/10
250/250 [==============================] - 7s 30ms/step - loss: 0.8730 - mean_absolute_error: 0.7012 - val_loss: 7.1169 - val_mean_absolute_error: 2.1313
Epoch 7

In [248]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

# Maximum number of words to consider
max_words = 1024  

tokenizer_1024 = Tokenizer(num_words=max_words)
tokenizer_1024.fit_on_texts(X_train.cleaned_descriptions)
desc_1024 = tokenizer_1024.texts_to_sequences(X_train.cleaned_descriptions)
max_seq_length = max(len(seq) for seq in desc_1024)

desc_1024_pad = pad_sequences(desc_1024, truncating='post', maxlen=max_seq_length)
text_input_1 = Input(shape=(max_seq_length,))

tokenizer_1024_test = Tokenizer(num_words=max_words)
tokenizer_1024_test.fit_on_texts(X_test.cleaned_descriptions)
desc_1024_test = tokenizer_1024_test.texts_to_sequences(X_test.cleaned_descriptions)
max_seq_length_test = max(len(seq) for seq in desc_1024_test)

desc_1024_pad_test = pad_sequences(desc_1024_test, truncating='post', maxlen=max_seq_length_test)

# winery
cat_input_1 = Input(shape=(1,))
embedding_1 = Embedding(input_dim=4720, output_dim=16)(cat_input_1)
flat_embed_1 = Flatten()(embedding_1)

# variety
cat_input_2 = Input(shape=(1,))
embedding_2 = Embedding(input_dim=306, output_dim=16)(cat_input_2)
flat_embed_2 = Flatten()(embedding_2)

# country
cat_input_3 = Input(shape=(1,))
embedding_3 = Embedding(input_dim=34, output_dim=16)(cat_input_3)
flat_embed_3 = Flatten()(embedding_3)

# year
cat_input_4 = Input(shape=(1,))
embedding_4 = Embedding(input_dim=28, output_dim=16)(cat_input_4)
flat_embed_4 = Flatten()(embedding_4)

embedding_dim = 128
embedding_1_text = Embedding(input_dim=max_words, output_dim=embedding_dim)(text_input_1)
flatten_1 = Flatten()(embedding_1_text)

concatenated = Concatenate()([flatten_1, flat_embed_1, flat_embed_2, flat_embed_3, flat_embed_4])
x = Dense(256, activation='relu')(concatenated)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='linear')(x)

model = Model(inputs=[text_input_1, cat_input_1, cat_input_2, cat_input_3, cat_input_4], outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=MeanSquaredError(),
              metrics=[MeanAbsoluteError()])

model.fit([desc_1024_pad, X_train['winery'], X_train['variety'], X_train['country'], X_train['year']], y_train,
          batch_size=32,
          epochs=10,
          validation_data=([desc_1024_pad_test, X_test['winery'], X_test['variety'], X_test['country'], X_test['year']], y_test))

loss, mae = model.evaluate([desc_1024_pad_test, X_test['winery'], X_test['variety'], X_test['country'], X_test['year']], y_test)
print('Test Loss:', loss)
print('Test MAE:', mae)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node model_16/embedding_65/embedding_lookup defined at (most recent call last):
  File "C:\Users\mor\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main

  File "C:\Users\mor\Anaconda3\lib\runpy.py", line 87, in _run_code

  File "C:\Users\mor\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\mor\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance

  File "C:\Users\mor\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start

  File "C:\Users\mor\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start

  File "C:\Users\mor\Anaconda3\lib\asyncio\base_events.py", line 601, in run_forever

  File "C:\Users\mor\Anaconda3\lib\asyncio\base_events.py", line 1905, in _run_once

  File "C:\Users\mor\Anaconda3\lib\asyncio\events.py", line 80, in _run

  File "C:\Users\mor\Anaconda3\lib\site-packages\ipykernel\kernelbas